In [ ]:
import pandas as pd

In [ ]:
music = pd.read_csv("final_download.csv")

In [ ]:
music['full_name'] = music['music_name'] + ' ' + music['singer']

In [ ]:
music

In [ ]:
music['singer'] = music['singer'].apply(lambda x: [singer.strip() for singer in x.split(',')])

In [ ]:
music['lyrics'] = music['lyrics'].apply(lambda x:x.split())
music['release'] = music['release'].astype(str)

In [ ]:
music['release'] =music['release'].apply(lambda x: [x])
music['singer'] = music['singer'].apply(lambda x: [i.replace(" ","") for i in x])
music['lyrics'] = music['lyrics'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
music

In [ ]:
music['tags'] = music['singer'] + music['release'] + music['lyrics']

In [ ]:
df = music[['music_name', 'full_name', 'thumbnail', 'tags']]

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df['tags'] = df['tags'].apply(lambda x:" ".join(x)) 

In [ ]:
df['tags'] = df['tags'].apply(lambda x:x.lower()) 

In [ ]:
df

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
ps = PorterStemmer()

In [ ]:
def stem(text):
    y= []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [ ]:
df['tags'] = df['tags'].apply(stem)

In [ ]:
df['tags'][2]

In [ ]:
hinglish = stopwords.words('hinglish')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500, stop_words= hinglish)

In [ ]:
vectors = cv.fit_transform(df['tags']).toarray()

In [ ]:
len(cv.get_feature_names_out())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similarity[1])), reverse=True, key=lambda x:x[1])[1:6]

In [ ]:
def recommend(musicz):
    music_index = df[df['music_name'] == musicz].index[0]
    distances = similarity[music_index]
    music_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in music_list:
        print(df.iloc[i[0]].music_name)

In [ ]:
recommend('Ae Ri Mai Re')

In [ ]:
import pickle 
pickle.dump(df.to_dict(), open('Music-Dict.pkl','wb'))
pickle.dump(similarity, open('MusicSimilarityMatrix.pkl','wb'))